# Integration Portal

![](https://theinflammatorydisease.files.wordpress.com/2019/10/specialty-pharmacy-1.jpg)

This is technically a webpage...


# Salesforce -- The Patient Connecections Platform
## Customer Support Program Web-App

- Lilly customer support programs are designed to provide patient's with a great experience 



![](https://www.salesforce.com/content/dam/web/en_is/www/images/home/unified/emea-astro-device.png)

# Simple-Salesforce
## PCP REST API

This is a binder outlining how we can query data in the patient connections platform using rest APIs in order to analyze/outline business processes. This style of coding wil allow us to .... technically speaking .... deprecate salesforce as a software platform (ROI).

-- Automated processes require python code in order to operate... otherwise the automated process breaks. 
* automation can be written in any language... there are a lot of programming languages, I'm using python for NLP. If you don't like python, read the english bit. We will need python in order to integrate with AWS appropriately, which is why we are using it.


## Establishing a connection to the server, must run before you attempt to run any other chunk of code!

This first code block is going to walk through how we can gain access to the data within salesforce to start writing queries. First you will need an account for the salesforce platform that you are trying to access. 

After you get your account set up, you should store your credentials securely at the operating system level. We'll import the operating system and then we can import a python package that will let us control salesforce programatically

## Specialty Pharmacy Portal (IQVIA)

IQVIA develops a webapp that provides us with valueable data. We use that data for automated metric reporting... which is why I'm using a notebook.

- Running this first chunk of code allows you to connect into our server if you have your salesforce credentials stored as environment variables: 
- since this is actually a django server, I can create HTML pages and post them... you can have a login to the web pages that does not give you access to this code too

The portal is integrated with our Patient Connections Platform (PCP)
-- it is a force.com platform, we can deprecate the entire platform with django -- cost savings

In [2]:
# ------------------------------- #
# imports 
# ------------------------------- #

import os
from simple_salesforce import Salesforce

# defining our log in credentials, saved as environment variables
username = os.environ.get('FORCE_PROD_USER')
password = os.environ.get('FORCE_PROD_PW')
security_token = os.environ.get('FORCE_PROD_TOKEN')


# ------------------------------- #
# imports 
# ------------------------------- #

def connect_to_prod(use_proxies):
    """ proxies give us elevated security during our connections sessions """
    if use_proxies:
        proxies = {
            'http': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000',
            'https': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000'
        }

        sf = Salesforce(username=username, password=password, security_token=security_token, proxies=proxies)
    else:
        sf = Salesforce(username=username, password=password, security_token=security_token)
    return sf

# call the connection to see if we can access data
connect_to_prod(True)

# SP Portal Automation Hot Fix -- Dispatch, PRODUCTION IS BROKEN

Establishing a connection to force.com in order to live-monitor the queue of cards recieved from IQVIA

-- this is automation, I have setup a django application connected to deldev3 that I want to use for testing the integration in a way that would allow us deploy code that interacts with the production application.

-- this notebook allows us to just play with data that is in production (the production code is currently broken at IQVIA)

First we setup a support function that will take in a bit of SOQL code from the PCP.

## Setting up a function to write SOQL queries

I wrote the connection into salesforce as a function, this way it is reusable. This specific function calls a connection into our production env.. kinda risky... but don't do anything stupid and you'll be fine, we're just repoting on data not manipulating it yet.

To perform salesforce development tasks, we should really use a connected app architecture and connect into a development environment. This connected app architecture is built through Django... and is a lot more... advanced...

For now I am just going to show off how to query data in order to write processes and conduct reports. I am going to make another reusable function so that writing queries is easier in the future. We can call the function by passing in our salesforce connection and providing a "SOQL" statement. SOQL is salesforce's modified version of SQL that is built specifically for the FORCE.com platform.


In [3]:
def query(sf, soql):
    # uses query_all so results are not paginated...
    soql_query = sf.query_all(soql)
    
    size = soql_query['totalSize']
    done = soql_query['done']
    # records are the actual data returned inside of the function's json
    records = soql_query['records']
    """ i am reformatting the return result so that this custom function just gives a clean python list """
    returned_records = []
    for record in records:
        record_fields = {}
        attributes = record['attributes']
        for key in record.keys():
            if record[key] != attributes:
                record_fields[key] = record[key]
        returned_records.append(record_fields)
    # cleaned python list of records, each record will have JSON to hold data
    return returned_records

# Monitor Each Record!

-- We are creating these as registrations right now, in deldev3 I have created a new object to start storing these records (where we need to be creating them).

-- I am developing a django application, which is a different framework than the jupyter notebook used here. That framework is production ready for release-related changes and testing. Django allows us to manage a server... and steal Salesforce's product for our own use. With Django we can also consolidate storage of technical assets, if it connects to the internet, it can connect to Django.

https://www.djangoproject.com/

# COMPLEX!!!!!!!
## Scripting

-- This code chunk is modeled after a django style "script". Just hit run to see each new update recieved. We can parse through the data by cleaning it with new requirements.... requirement gathering.

In [4]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"



# return the results of the query in a list using our custom function
docs = query(sf, soql)


def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    
    # return the map
    return copay_card_map


def parse_copay_map(new_copay_cards_recieved):
    for card in new_copay_cards_recieved:
        print(card)


def clean_copay_map(copay_map):
    """ allows us to parse out the registrations and documents that are attached to a specific savings card id """
    first_card = list(copay_map.keys())[0]
    
    # allows us access to new registrations associated with the card... they aren't registrations, they are status updates
    registrations = copay_map[first_card][0]
    # documents are created in SFDC
    documents = copay_map[first_card][1]
    
    # cleaning the data to read as a status update (the way I loaded it in my django script...)
    for status_update in registrations:
        # show the data structure... its coverage
        for update in status_update:
            status_data = status_update[update]
            print(update, '\t', status_data)
            # stop gap to next update
            input("\npress enter to see next update...")
            print("")


# ------------------------------------------------------------------- #
# scripts in the django format require a 'run' similar to a 'main'
# ------------------------------------------------------------------- #

def run():
    test_map = group_documents_by_copay_id(docs)
    # parse_copay_map(test_map)
    clean_copay_map(test_map)


# run the script
run()


attributes 	 OrderedDict([('type', 'DTPC_Registration__c'), ('url', '/services/data/v38.0/sobjects/DTPC_Registration__c/a1H4P000007OxOUUA0')])

Hub_Patient_ID__c 	 P-0000877415

PA_Submitted__c 	 Yes

PA_Submitted_Date__c 	 2021-04-15

PA_Outcome__c 	 None

PA_Denial_Reason__c 	 None

PA_Expired__c 	 None

Appeal_Submitted__c 	 None


KeyboardInterrupt: Interrupted by user

## List queries

this support function will allow us to do more interesting queries... you'll see it in action shortly, but it will be allow us to write queries that take in a list of specific values and return data connected to that specific list of records. For example, if I want to look at 10 specific records and return related record data specific to only those 10 records, we would call this function.

In [5]:
def query_list(sf, soql, the_list):
    """ when using the IN operator, if the list contains more than 20k characters, it hits a limit so i'll partition the query using chunks of length 700 """
    records = []
    number_of_lists = round(len(the_list) / 700)
    if number_of_lists == 0:
        number_of_lists = 1
    start_index = 0
    try:
        partition_length = round(len(the_list) / number_of_lists)
    except:
        partition_length = round(len(the_list))
    end_index = partition_length
    for i in range(number_of_lists):
        sub_list = the_list[start_index:end_index]
        list_string = format_list(sub_list)
        sub_soql = soql + "%s" % list_string
        try:
            sf_data = query(sf, sub_soql)
        except ValueError:
            sf_data = []
            print("SOQL error, no data returned")
        for record in sf_data:
            records.append(record)
        start_index = end_index
        end_index = partition_length * (i + 2)
    # returning all the records found in the query
    return records

## The PCP program object

The salesforce application that we are connected to is called the 'Patient Connections Platform'. This app is built off of a series of totally custom objects and workflows. The core object of the app is called the 'Program'.

A program record represents a single patient's enrollment into a Customer Support Program. I've created a bunch of utility functions that will outline the data model for a program and allow us to quickly query records in the PCP. Reading through this will help you understand the related objects to a CSP patient's program.

*remember, these functions are designed to abstract away complexity, i am showing off the usage of the simple-salesforce package through a notebook but structuring these functions through a file directory will allow you to make them reusable*

In [6]:
# --------------------------------- #
# consent parsing
# --------------------------------- #

def parse_consents(program):
    """ consents are processed upon patient enrollment into a CSP and stored to either a patient/caregiver through
     the program record """
    try:
        consents = program['Consent_Auth_Preferences__r']['records']
        consent_data = []
        for record in consents:
            consent_record = {}
            for key in record:
                if key != 'attributes':
                    consent_record[key] = record[key]
            consent_data.append(consent_record)
    except KeyError:
        consent_data = []

    # return a list of all the consent records cleaned up
    return consent_data


# --------------------------------- #
# coverage parsing
# --------------------------------- #

def parse_coverages(program):
    """ takes in a program and returns a list containing a list of copay coverages and a list of insurance coverages """
    try:
        # store copay records in first list, insurance records in the second list
        coverage_data = [[], []]
        coverages = program['Coverage1__r']['records']
        for record in coverages:
            try:
                if record['RecordTypeId'] == '012f20000001HL3AAM':
                    copay_coverage_record = {}
                    for key in record:
                        if key != 'attributes' and key != 'RecordTypeId':
                            copay_coverage_record[key] = record[key]
                    coverage_data[0].append(copay_coverage_record)

                elif record['RecordTypeId'] == '012f20000001HL4AAM':
                    insurance_coverage_record = {}
                    for key in record:
                        if key != 'attributes' and key != 'RecordTypeId':
                            insurance_coverage_record[key] = record[key]
                    coverage_data[1].append(insurance_coverage_record)

            except KeyError:
                print("queries that incorporate coverage records must include the RecordTypeId field")

    # return an empty data model if we pass the method an invalid data structure
    except KeyError:
        coverage_data = [[], []]
    return coverage_data


def filter_active_coverages(coverages):
    """ takes a list of coverage records and parses out any inactive records (good for separating ins and copay) """
    active_coverages = []
    for coverage in coverages:
        try:
            if coverage['DTPC_Record_Status__c'] == 'Active':
                active_coverages.append(coverage)
        except KeyError:
            print("this method requires you to pull the DTPC_Record_Status__c from coverage")
    return active_coverages


def parse_old_coverages(program):
    try:
        # store copay records in first list, insurance records in the second list
        coverage_data = []
        coverages = program['Coverage__r']['records']
        for record in coverages:
            old_coverage_record = {}
            for key in record:
                if key != 'attributes':
                    old_coverage_record[key] = record[key]
            coverage_data.append(old_coverage_record)
    # return an empty data model if we pass the method an invalid data structure
    except KeyError:
        coverage_data = []
    return coverage_data


def filter_active_old_coverages(coverages):
    active_coverages = []
    for coverage in coverages:
        try:
            if coverage['Coverage_Status__c'] == 'Active':
                active_coverages.append(coverage)
        except KeyError:
            print("this method requires your query to pull the Coverage_Status__c field from coverage")
    return active_coverages


# --------------------------------- #
# claims parsing
# --------------------------------- #

def parse_claims(program):
    try:
        claims = program['Claims_Adjudication__r']['records']
        claims_data = []
        for record in claims:
            claim_record = {}
            for key in record:
                if key !='attributes':
                    claim_record[key] = record[key]
            claims_data.append(claim_record)
    except KeyError:
        claims_data = []

    return claims_data


# --------------------------------- #
# dispense parsing
# --------------------------------- #

def parse_dispenses(program):
    try:
        dispenses = program['Specialty_Pharmacy_Dispenses__r']['records']
        dispense_data = []
        for record in dispenses:
            dispense_record = {}
            for key in record:
                if key != 'attributes':
                    dispense_record[key] = record[key]
            dispense_data.append(dispense_record)
    except KeyError:
        dispense_data = []

    return dispense_data


# --------------------------------- #
# program services parsing
# --------------------------------- #

def parse_program_services(program):
    try:
        services = program['Program_Services__r']['records']
        services_data = []
        for record in services:
            service_record = {}
            for key in record:
                if key != 'attributes':
                    service_record[key] = record[key]
            services_data.append(service_record)
    except KeyError:
        services_data = []

    return services_data


# --------------------------------- #
# agent workflow parsing
# --------------------------------- #

def parse_cases(program):
    try:
        cases = program['Cases__r']['records']
        case_data = []
        for record in cases:
            case_record = {}
            for key in record:
                if key != 'attributes':
                    case_record[key] = record[key]
            case_data.append(case_record)
    except KeyError:
        case_data = []

    return case_data


def parse_document_logs(program):
    logs = program['Document_Logs__r']
    try:
        document_logs = logs['records']
        document_data = []
        for record in document_logs:
            document_record = {}
            for key in record:
                if key != 'attributes':
                    document_record[key] = record[key]
            document_data.append(document_record)

    except KeyError:
        document_data = []

    # return the two separate data structures
    return document_data

## Process based reporting

We are now going to use all of the functions that I just shared to show off how we can construct different types of reports that access data in the PCP. This specific notebook will monitor the inbound records recieved via IQVIA for the SP Portal.

## SP Portal

This process is going to analyze data that was loaded into salesforce from IQVIA, the data is manually processed by call center agents today... we are working on automating that...

In [8]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"

# return the results of the query in a list using our custom function
docs = query(sf, soql)

# print out the number of docs our query returned
print(len(docs))

256


In [9]:

# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"



# return the results of the query in a list using our custom function
docs = query(sf, soql)


def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    
    # return the map
    return copay_card_map

test_map = group_documents_by_copay_id(docs)
print(type(test_map))

<class 'dict'>


## Evaluate New Inbound Copay Cards
*Simple function will print out all of the card ids that have recieved a new update*

This will evaluate all of the new cards that were recieved.

In [10]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"



# return the results of the query in a list using our custom function
docs = query(sf, soql)


def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    
    # return the map
    return copay_card_map


def parse_copay_map(new_copay_cards_recieved):
    for card in new_copay_cards_recieved:
        print(card)


# ------------------------------------------------------------------- #
# scripts in the django format require a 'run' similar to a 'main'
# ------------------------------------------------------------------- #

def run():
    test_map = group_documents_by_copay_id(docs)
    parse_copay_map(test_map)


# run the script
run()

E46103092427
E46103134217
E49102652268
E46103134143
E46102180607
E46102901395
E49102683835
E46103137508
E46102926948
E46102910848
E46102888448
E46103141635
E46103128474
E46102870312
E49102699377
E49102647221
E46103082063
E46102145184
E46103102502
E46101489073
E49101922865
E46103033133
E49101260310
E46101757385
E46103104390
E49102698047
E46101126610
E46103101452
E46103012694
E46102522555
E46103112513
E46103113983
E49102623704
E46102863108
E46103143736
E49102662763
E49102003573
E46101176591
E49102670533
E49102659826
E49102693213
E46103127070
E46103143243
E46102462351
E49102694822
E49102698112
E49102686000
E49102680125
E49102680330
E49102574841
E49102609353
E49100387838
E46103094873
E46103111184
E46103107401
E46103007163
E49101821641
E49102661926
E49102680963
E46102864857
E46103142686
E49102692861
E49102680618
E49102666193
E46103091935
E49102396347
E49101783774
E46103097950
E46101951280
E46102304855
E46103067992
E46103111956
E49102693008
E46103149472
E49102697908
E46103001632
E46102950045

## Data Cleaning
*parse the new card values for update types*

Use a Linux server instead of informatica, build out in deldev3